In [38]:
import pandas as pd
import ezodf

def read_ods_by_name(file_path, sheet_name):
    ods = ezodf.opendoc(file_path)
    
    sheet = ods.sheets[sheet_name]
    
    data = []
    for row in sheet.rows():
        data_row = [cell.value for cell in row]
        data.append(data_row)
    
    df = pd.DataFrame(data)
    return df


Reading the TM'S

In [39]:
file_path = 'SPORT_documents//sport_ttc_20220814.ods'

sheets = ['TM_CDH_1', 'TM_CDH_2', 'TM_PLD', 'TM_DSU', 'TM_ACS', 'TM_SCI', 'TM_HOU', 'TM_STD', 'TM_BEA']

columns = ['identification','name','version_number', 'pkt_type', 'sec_hdr_flag', 'apid', 'seq_flags', 'seq_count', 'pkt_data_length', 'secondary_header', 'data', 'checksum']

main_tm_df = pd.DataFrame()

for sheet_name in sheets:
    df = read_ods_by_name(file_path, sheet_name)
    df = df.iloc[6:, 0:12].dropna(axis = 0, how = 'all')

    main_tm_df = pd.concat([main_tm_df, df])

main_tm_df.columns = columns

main_tm_df

,identification,name,version_number,pkt_type,sec_hdr_flag,apid,seq_flags,seq_count,pkt_data_length,secondary_header,data,checksum
6,TM_CDH_001,C&DH: get time,0.0,0.0,1.0,0x201,0x3,0.0,0x1A,Time,DD_CDH_001,CRC-16
7,TM_CDH_005,TT&C: get latest TC headers,0.0,0.0,1.0,0x205,0x3,0.0,0x4F,Time,DD_CDH_005,CRC-16
8,TM_CDH_006,TT&C: get latest standard TC response,0.0,0.0,1.0,0x206,0x3,0.0,0x1E,Time,DD_CDH_006,CRC-16
9,TM_CDH_011,TT&C: initiate crypto key exchange,0.0,0.0,1.0,0x20B,0x3,0.0,0x29,Time,DD_CDH_011,CRC-16
10,TM_CDH_012,TT&C: validate candidate crypto key (pt. 1),0.0,0.0,1.0,0x20C,0x3,0.0,0x11,Time,DD_CDH_012,CRC-16
...,...,...,...,...,...,...,...,...,...,...,...,...
49,TM_HOU_021_C,HK ADCS_OBS_NM_EXT (compressed),0.0,0.0,1.0,0x03D,0x3,Varies,Variable,Time,DC_HOU_OBS_4E,CRC-16
50,TM_HOU_022_C,HK ADCS_OBS_SPM_EXT (compressed),0.0,0.0,1.0,0x03E,0x3,Varies,Variable,Time,DC_HOU_OBS_5E,CRC-16
51,TM_HOU_023_C,HK ADCS_OBS_MICM_EXT (compressed),0.0,0.0,1.0,0x03F,0x3,Varies,Variable,Time,DC_HOU_OBS_6E,CRC-16
6,TM_STD_001,Standard uplink frame response,0.0,0.0,1.0,0x001,0x3,Varies,0xE,Time,Varies,ErrCode


Reading the DD's

In [40]:
file_path = 'SPORT_documents//sport_ttc_20220814.ods'
sheet_name = 'DD_CDH'  
df = read_ods_by_name(file_path, sheet_name)

df.iloc[:, 1:8].dropna(axis = 0, how = 'all')


,1,2,3,4,5,6,7
1,DD_CDH_001,None,None,None,None,None,None
2,Field,Len\n(bits),Format,Nominal\nminimum,Nominal\nmaximum,None,None
3,GPS time (C&DH view),64.0,GPS time,N/A,N/A,None,None
4,Free-running clock,64.0,GPS time,N/A,N/A,None,None
5,GPS time origin,8.0,uint8_t,N/A,N/A,None,"0 is FRC, 1 is GROUND, 2 is CTECS, 3 is CTECS ..."
...,...,...,...,...,...,...,...
433,EFP2 deployed,1.0,bit,N/A,N/A,None,"0 is DEPLOYED, 1 is NOT DEPLOYED"
434,EFP2 stowed,1.0,bit,N/A,N/A,None,"0 is STOWED, 1 is NOT STOWED"
435,EFP1 deployed,1.0,bit,N/A,N/A,None,"0 is DEPLOYED, 1 is NOT DEPLOYED"
436,EFP1 stowed,1.0,bit,N/A,N/A,None,"0 is STOWED, 1 is NOT STOWED"
